# Your favorite word embedding

<font color='red'>
    <br>
Put in the cells below your favorite "vectorized word embedding" and submit your notebook in the lms (word embedding section before midnight Thursday the 3rd).
</font>

In [ ]:
"""
(Practical tip) Table of contents can be compiled directly in jupyter notebooks using the following code:
I set an exception: if the package is in your installation you can import it otherwise you download it 
then import it.
"""
try:
    from jyquickhelper import add_notebook_menu 
except:
    !pip install jyquickhelper
    from jyquickhelper import add_notebook_menu
    
"""
Output Table of contents to navigate easily in the notebook. 
For interested readers, the package also includes Ipython magic commands to go back to this cell
wherever you are in the notebook to look for cells faster
"""
add_notebook_menu()

## Imports

In [ ]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model
from keras.layers import Input, TextVectorization, Dense, Dropout, Flatten, Embedding

!pip install tensorflow-addons

import tensorflow_addons as tfa
from tensorflow_addons.metrics import F1Score

In [ ]:
!pip install gensim
from gensim.test.utils import datapath
from gensim import utils
import gensim.models

## Dataset

In [ ]:
TRAIN = pd.read_csv("http://www.i3s.unice.fr/~riveill/dataset/Amazon_Unlocked_Mobile/train.csv.gz")
VAL = pd.read_csv("http://www.i3s.unice.fr/~riveill/dataset/Amazon_Unlocked_Mobile/train.csv.gz")
TEST = pd.read_csv("http://www.i3s.unice.fr/~riveill/dataset/Amazon_Unlocked_Mobile/test.csv.gz")

In [ ]:
''' Construct X_train and y_train '''
X_train = TRAIN['Reviews']
y_train = np.array(TRAIN['Rating']).reshape(-1,1)

X_val = VAL['Reviews']
y_val = np.array(VAL['Rating']).reshape(-1,1)

X_test = TEST['Reviews']
y_test = np.array(TEST['Rating']).reshape(-1,1)


ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
y_train_enc = ohe.fit_transform(y_train)
y_val_enc = ohe.fit_transform(y_val)
y_test_enc = ohe.fit_transform(y_test)

X_train.shape, y_train_enc.shape, np.unique(y_train)

## Word embedding and MLP classifier

### Constant definition


<font color='red'>
Use these constant names, you can of course change the value.
</font>

In [ ]:
vocab_size = 500 # Vocab size
max_len = 100   # Sequence length to pad the outputs to.
embed_dim = 100 # embedding dimension
dense_dim = 128 # dense dimension
dropout_rate = 0.5 # Dropout rate
nb_classes = len(np.unique(y_train))

### Define Embedding Layer

<font color='red'>
$TO DO - Students$

* Define your embedding matrix or your vectorizer layer
    * --> if you choose Keras embedding -- vectorizer layr
    * --> if you choose Word2Vec, Glove or FastTex -- embedding matrix
    * --> if you choose gensim -- build you gensim model and then the embedding matrix
</font>

In [ ]:
""" FILL """

### Build, fit and evaluate the network

<font color='red'>
$TO DO - Students$

Using your previous layer build a very simple MLP with:
* one hidden layer Dense + Dropout cells
    
print the summary of your model
</font>

In [ ]:
# define the model
""" FILL """

model = Model(input_, output_)
# summarize the model
model.summary()

<font color='Blue'>
* Compile the model with F1 score.
</font>

In [ ]:
# compile the model
f1 = F1Score(num_classes=nb_classes, average="macro", threshold=0.5)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[f1])

<font color='blue'>
* Fit the modele using early stopping on f1_score
</font>

In [ ]:
# fit model using ealy stopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_f1_score', mode='max', patience=5, restore_best_weights=True, verbose=1)
history = model.fit(X_train, y_train_enc,
                    validation_data=[X_val, y_val_enc],
                    epochs=4000, callbacks=[es], verbose=1)

<font color='blue'>
* Babysit your model
</font>

In [ ]:
# Babysit the model
pd.DataFrame({'val_loss':history.history['val_loss'],
              'loss':history.history['loss'],
             'val_f1_score':history.history['val_f1_score'],
              'f1_score':history.history['f1_score']}).plot(figsize=(8,5))

<font color='blue'>
* Babysit your model
</font>

In [ ]:
# Evaluate the model
f1.update_state(y_test_enc, model.predict(X_test))
result = f1.result()
result.numpy()